In [388]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

In [402]:
#functions

def rand(scale):
    return scale*(np.random.rand()+np.random.rand()*1j-0.5-0.5j)

def randmatrix(scale):
    a=rand(scale)
    b=rand(scale)
    c=rand(scale)
    d=rand(scale)
    e=rand(scale)
    f=rand(scale)
    g=rand(scale)
    h=rand(scale)
    M=np.array([[0,0,a,b,c],[0,0,d,e,f],[a.conjugate(),d.conjugate(),0,0,g],[b.conjugate(),e.conjugate(),0,0,h],[c.conjugate(),f.conjugate(),g.conjugate(),h.conjugate(),0]])
    return(M)

def randomw():
    d=5
    w=np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    w=np.sort(w)
    return(w-sum(w)/d)

def conjidregime(w):
    return(0>(w[1]+w[2])/2 and 0<(w[2]+w[3])/2 and 0>(w[0]+w[3])/2 and 0<(w[1]+w[4])/2)

def newconjidregime(w): # added "and w[1]<0 and w[3]>0" in return
    return(0>(w[1]+w[2])/2 and 0<(w[2]+w[3])/2 and 0>(w[0]+w[3])/2 and 0<(w[1]+w[4])/2 and w[1]<0 and w[3]>0)

In [215]:
#testing functions

M=randmatrix(0.5)
#M.T.conjugate()-M
print(M.round(2), "\n" )
w=np.linalg.eigvalsh(M)
print(w)

#checking linalg.eig. Columns of the unitary np.linalg.eig(M)[1] are the eigenvectors of M

evector=(np.linalg.eig(M)[1]).T[2]
evalue=np.linalg.eig(M)[0][2]
print((evalue*evector - np.matmul(M,evector) ).round(15))
U=np.linalg.eig(M)[1]
np.dot(U,U.T.conjugate()).round(15)

[[ 0.  +0.j    0.  +0.j   -0.17+0.12j  0.  +0.18j  0.09-0.18j]
 [ 0.  +0.j    0.  +0.j    0.07+0.23j  0.1 -0.21j -0.19+0.24j]
 [-0.17-0.12j  0.07-0.23j  0.  +0.j    0.  +0.j    0.16+0.1j ]
 [ 0.  -0.18j  0.1 +0.21j  0.  +0.j    0.  +0.j   -0.11+0.19j]
 [ 0.09+0.18j -0.19-0.24j  0.16-0.1j  -0.11-0.19j  0.  +0.j  ]] 

[-0.56161896 -0.22075349 -0.02098647  0.16239757  0.64096135]
False 

[-0.+0.j  0.+0.j  0.+0.j  0.-0.j -0.+0.j]


array([[ 1.+0.j,  0.-0.j,  0.-0.j, -0.-0.j, -0.+0.j],
       [ 0.+0.j,  1.+0.j, -0.+0.j,  0.-0.j, -0.-0.j],
       [ 0.+0.j, -0.-0.j,  1.+0.j, -0.-0.j,  0.-0.j],
       [-0.+0.j,  0.+0.j, -0.+0.j,  1.+0.j,  0.+0.j],
       [-0.-0.j, -0.+0.j,  0.+0.j,  0.-0.j,  1.+0.j]])

In [307]:
# start to test conjectures 

maxcount=1000000
count=0
countcounterex=0
scale=0.5
eps=0.0 #for small values you should always find counterexamples

while count<maxcount:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        count=count+1
        if 0<(w[1]+w[2])/2+eps or 0>(w[2]+w[3])/2-eps:
            countcounterex=countcounterex+1
print("found ",countcounterex, " counterexamples")

found  0  counterexamples


In [245]:
# an idea i have is that if w[0] and w[1] and/or w[3] and w[4] are too close e.g. wrt diameter,
#then max mixed v is not in the v-set
#notice that if w[0], w[1] are close and w[3], w[4] are close then (w[1]+w[2])/2 < 0 < (w[2]+w[3])/2 follows wathever w[2] is

maxcount=1000000
count=0
countcounterex=0
scale=0.5
ratio=0.04

while count<maxcount:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        count=count+1
        if w[1]-w[0]<ratio*diameter and w[4]-w[3]<ratio*diameter:
            countcounterex=countcounterex+1
            print(w)
            print((w[0]+w[3])/2<0)
            print((w[1]+w[4])/2>0)
print("found ",countcounterex, " counterexamples")

[-0.35106016 -0.32999979  0.00477769  0.32488581  0.35139645]
True
True
[-0.4083248  -0.37928511  0.02652062  0.364864    0.39622529]
True
True
[-0.44152814 -0.42208167 -0.00299026  0.42105775  0.44554231]
True
True
[-0.48463292 -0.45153442  0.0284852   0.44503238  0.46264976]
True
True
[-0.40674303 -0.38638513 -0.0076374   0.38588198  0.41488358]
True
True
[-0.33894206 -0.31380022 -0.00849663  0.32318656  0.33805235]
True
True
[-0.40812977 -0.38794511  0.00453277  0.38738023  0.40416187]
True
True
[-0.4749514  -0.43981248  0.01266845  0.4333182   0.46877723]
True
True
[-0.43864409 -0.40937903  0.02036031  0.41084947  0.41681334]
True
True
found  9  counterexamples


In [292]:
# ok based on previous box this could be the new inequalities!!

maxcount=1000000
count=0
countcounterex=0
scale=0.4
eps=0.001 #for small values you should always find counterexamples

while count<maxcount:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        count=count+1
        if 0<(w[0]+w[3])/2+eps or 0>(w[1]+w[4])/2-eps:
            countcounterex=countcounterex+1
print("found ",countcounterex, " counterexamples")

found  5  counterexamples


In [230]:
# try to understand form of eigenvectors... patterns?

M=randmatrix(scale)
w=np.linalg.eigvalsh(M)
if matfilter(w):
    U=np.linalg.eig(M)[1]
    print(w,"\n")
    print(U.round(2))


[-0.45121596 -0.32996769  0.02132308  0.2765454   0.48331517] 

[[-0.03+0.14j  0.7 +0.j   -0.31-0.02j  0.32-0.25j  0.01+0.47j]
 [ 0.62+0.j   -0.02+0.1j  -0.33-0.08j -0.11-0.48j -0.39-0.32j]
 [ 0.18-0.4j  -0.2 +0.39j -0.38+0.17j  0.28+0.02j  0.59+0.j  ]
 [-0.3 -0.28j -0.27-0.48j -0.35+0.03j  0.52+0.j   -0.36-0.11j]
 [ 0.34-0.34j -0.  -0.05j  0.7 +0.j    0.44-0.22j -0.05+0.17j]]


In [306]:
# check conjecture on identity regime boundaries, first direction

maxcount=1000000
count=0
countcounterex=0
scale=0.4
eps=0.001
epsvec=np.array([eps,eps,eps,eps,eps])

while count<maxcount:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)+epsvec
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        count=count+1
        if conjidregime(w)==False:
            countcounterex=countcounterex+1
print("found ",countcounterex, " counterexamples")

found  17  counterexamples


In [399]:
# check conjecture on identity regime boundaries, second direction. generate random w, check conjidregime,
#and if true try to get close to it with spectra of randmatrix matrces. three steps

#first step: generate numerical set

#maxcount=1000
count=0
#countcounterex=0
scale=0.5
numtruepoints=1000000

numset=[]
while count < numtruepoints:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        numset.append(w)
        count=count+1

In [397]:
# second step: generate a smaller conjectured set

#countcounterex=0
scale=0.5
numconjpoints=10
distance=0.
count=0

#generate numerical set
conjset=[]
while count < numconjpoints:
#control run. by filling conjset with  vectors which actually are in numset,
#the list distances in next box is  filled with zeros provided we are doing large enough numbers
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        conjset.append(w)
        count=count+1

In [400]:
#actual run

count=0
conjset=[]
while count < numconjpoints:
    w=randomw()
    if newconjidregime(w)==True:
        conjset.append(w)
        count=count+1
#print(conjset)

In [401]:
#third step: check minimal distance of each  point of conjset from points of numset
dist=100.
newdist=100.
distances=[]
delta=np.zeros(5)
for j in range(numconjpoints):
    dist=100.
    for k in range(numtruepoints):
        delta = conjset[j]-numset[k]
        newdist = np.linalg.norm(delta)
        if newdist<dist:
            dist=newdist
    distances.append(dist)
print(distances)

[0.004972410635939002, 0.0042671907925267705, 0.009272291588375772, 0.003579655573170788, 0.03172745145674119, 0.02811984883049278, 0.03877116751070378, 0.028267537369379817, 0.011764589431356743, 0.015270179009059805]


In [361]:
distances[3]

0.11811862879421998

In [373]:
dist=100.
newdist=100.
for k in range(numtruepoints):
    delta = critical-numset[k]
    newdist = np.linalg.norm(delta)
    if newdist<dist:
        dist=newdist
print(dist)

0.13261994427112764


In [375]:
print(critical)

[-0.12366249 -0.07573321 -0.04888961  0.10446614  0.14381918]


In [377]:
conjidregime(critical)

True

In [383]:
scale=0.4
count=0
dist=100.
numtruepoints=500000
while count < numtruepoints:
    M=randmatrix(scale)
    w=np.linalg.eigvalsh(M)
    diameter=w[4]-w[0] #otherwise it s not pauli compatible
    if diameter<1:
        newdist = np.linalg.norm(delta)
        if newdist<dist:
            dist=newdist
print(dist)

KeyboardInterrupt: 

In [387]:
#a nasty w vector. i dont understand if it belongs to id regime. or are there more regimes?
criticalw = np.array([0.07633751, 0.12426679, 0.15111039, 0.30446614, 0.34381918])

In [385]:
dist

0.09930461943197591

In [411]:
criticalw = np.array([0.07633751, 0.12426679, 0.15111039, 0.30446614, 0.34381918])
crit=criticalw-sum(criticalw)/5
crit

array([-0.12366249, -0.07573321, -0.04888961,  0.10446614,  0.14381918])